# L4: 고객의 행사 관련 협업을 제안하는 서비스를 만듭니다.
> Tool의 세 가지 핵심 요소에 초점을 맞출 것입니다:
- 다용도성
- 내부 결함성
- Cashing

In [21]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- 라이브러리, API 및 LLM 가져오기- 서퍼](https://serper.dev)

In [22]:
from crewai import Agent, Task, Crew

In [23]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
import os
load_dotenv()

gpt = ChatOpenAI(model= "gpt-3.5-turbo", api_key=os.getenv("OPENAI_API_KEY"))

## 에이전트 만들기
- sales_rep_agent : 고객에 맞는 솔루션 제공
- lead_sales_rep_agent : 대화를 리드

In [24]:
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match "
         "our ideal customer profile",
    backstory=(
        "As a part of the dynamic sales team at CrewAI, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    ),
    allow_delegation=False,
    verbose=True
)

In [25]:
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    verbose=True
)

## Tool 만들기
### Crew AI Tool

In [26]:
from crewai_tools import DirectoryReadTool,FileReadTool, SerperDevTool

In [27]:
directory_read_tool = DirectoryReadTool(directory='./instructions')
file_read_tool = FileReadTool()
search_tool = SerperDevTool(api_key = os.getenv("SERPER_API_KEY"))

### 사용자 지정 도구- 크루에이아이의 [BaseTool](https://docs.crewai.com/core-concepts/Tools/#subclassing-basetool) 클래스를 사용하여 사용자 지정 도구 만들기

In [28]:
from crewai_tools import BaseTool

- 모든 도구에는 '이름'과 '설명'이 있어야 합니다.- 간소화 및 수업 목적을 위해 `SentimentAnalysisTool`은 모든 텍스트에 대해 `긍정`을 반환합니다.- 로컬에서 실행할 때는 `_run` 함수에서 로직으로 코드를 사용자 정의할 수 있습니다.

In [29]:
class SentimentAnalysisTool(BaseTool):
    name: str ="Sentiment Analysis Tool"
    description: str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")
    
    def _run(self, text: str) -> str:
        # Your custom code tool goes here
        return "positive"

In [30]:
sentiment_analysis_tool = SentimentAnalysisTool()

## 작업 만들기
- 리드 프로파일링 작업 : directory_read_tool, file_read_tool, search_tool 사용

In [31]:
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)

- 개인화된  작업은 사용자 지정 도구인 'SentimentAnalysisTool'과 크루AI의 'SerperDevTool'(search_tool)을 사용하고 있습니다.

In [32]:
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)

## crew 생성하기

In [33]:
crew = Crew(
    agents=[sales_rep_agent, 
            lead_sales_rep_agent],
    
    tasks=[lead_profiling_task, 
           personalized_outreach_task],

    verbose=2,
    memory=True
)

2024-05-28 21:33:42,839 - 20496 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


## crew 운영 

In [36]:
inputs = {
    "lead_name": "Samsung Electronics Co.",
    "industry": "Electronics",
    "key_decision_maker": "No Tae Moon",
    "position": "CEO",
    "milestone": "Galaxy25 launch"
}

result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Sales Representative
 [INFO]: == Starting Task: Conduct an in-depth analysis of Samsung Electronics Co., a company in the Electronics sector that recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile, focusing on key decision-makers, recent business developments, and potential needs that align with our offerings. This task is crucial for tailoring our engagement strategy effectively.
Don't make assumptions and only use information you absolutely sure about.


> Entering new CrewAgentExecutor chain...
To begin my research on Samsung Electronics Co., I will need to gather as much information as possible about the company. This includes details about the company's background, key decision-makers, recent business developments, and identified needs. To do this, I will start by using the internet search tool to find these details.

Action: 
Search the internet

Action Input: 
{"search_query": "Samsung Electroni

- 최종 결과를 마크다운으로 표시합니다.

In [37]:
from IPython.display import Markdown
Markdown(result)

Here is the final draft of the outreach campaign:

Dear Mr. No Tae Moon,

I am writing to you from DeepLearningAI, a company dedicated to making world-class AI education accessible globally. We have closely followed Samsung's journey, and we are impressed by your commitment to agility, integrity, performance, respect, and excellence. Your dedication to constant change, ethical conduct, and creating a culture of mutual respect resonates with us.

We understand that Samsung is anticipating the launch of the Galaxy25 series in early 2025, and we believe this is an exciting opportunity for Samsung to further solidify its position as a global leader in technology. Our solutions, designed to support ongoing tech innovations and digital business transformations, could be an excellent fit for Samsung's vision for this launch.

Additionally, we acknowledge Samsung's commitment to environmental sustainability. Our philosophy aligns with yours in this regard, as we too prioritize creating superior products and services that contribute to a better global society. We would love to explore how our AI solutions could aid in Samsung's sustainability efforts.

We look forward to the possibility of collaborating with Samsung to achieve these shared goals. Please let us know if you would be interested in discussing this further.

Best Regards,
[Your Name]
Lead Sales Representative, DeepLearningAI

# L5: 이벤트 계획 자동화
- 이 단원에서는 작업에 대해 자세히 알아보세요.

In [48]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- 라이브러리, API 및 LLM 가져오기

In [49]:
from crewai import Agent, Crew, Task

In [50]:
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI

from dotenv import load_dotenv
import os
load_dotenv()

gpt = ChatOpenAI(model= "gpt-3.5-turbo", api_key=os.getenv("OPENAI_API_KEY"))
gemini = ChatGoogleGenerativeAI(model="gemini-pro",google_api_key=os.getenv("GEMINI_API_KEY"))
os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")

gpt = ChatOpenAI(model= "gpt-3.5-turbo", api_key=os.getenv("OPENAI_API_KEY"))
os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")

## 크루AI 도구

In [51]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# Initialize the tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## 에이전트 만들기

In [52]:
# Agent 1: Venue Coordinator
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue "
    "based on event requirements",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "With a keen sense of space and "
        "understanding of event logistics, "
        "you excel at finding and securing "
        "the perfect venue that fits the event's theme, "
        "size, and budget constraints."
    ),
    llm = gpt
    
)

In [53]:
 # Agent 2: Logistics Manager
logistics_manager = Agent(
    role='Logistics Manager',
    goal=(
        "Manage all logistics for the event "
        "including catering and equipmen"
    ),
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Organized and detail-oriented, "
        "you ensure that every logistical aspect of the event "
        "from catering to equipment setup "
        "is flawlessly executed to create a seamless experience."
    ),
    llm=gpt
    
)

In [54]:
# Agent 3: Marketing and Communications Agent
marketing_communications_agent = Agent(
    role="Marketing and Communications Agent",
    goal="Effectively market the event and "
         "communicate with participants",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Creative and communicative, "
        "you craft compelling messages and "
        "engage with potential attendees "
        "to maximize event exposure and participation."
    ),
    llm=gemini
)

## [Venue pydantic 오브젝트 생성- pydantic 베이스모델](https://docs.pydantic.dev/latest/api/base_model/)을 사용하여 `VenueDetails` 클래스를 생성합니다.
- 상담원은 이 객체의 여러 인스턴스를 생성하여 다양한 장소에 대한 정보로 이 객체를 채웁니다.

In [55]:
from pydantic import BaseModel
# Define a Pydantic model for venue details 
# (demonstrating Output as Pydantic)
class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str

## 작업 만들기- output_json`을 사용하면 원하는 출력의 구조를 지정할 수 있습니다.
- 출력_파일`을 사용하면 출력을 파일로 가져올 수 있습니다
.- '휴먼_입력=참'으로 설정하면 작업이 완료되기 전에 사람의 피드백(결과 마음에 드는지 여부)을 묻습니다.

In [56]:
venue_task = Task(
    description="Find a venue in {event_city} "
                "that meets criteria for {event_topic}.",
    expected_output="All the details of a specifically chosen"
                    "venue you found to accommodate the event.",
    human_input=True,
    output_json=VenueDetails,
    output_file="venue_details.json",  
      # Outputs the venue details as a JSON file
    agent=venue_coordinator
)

- 비동기 실행=False`를 설정하면 작업이 그 뒤에 오는 작업과 병렬로 실행될 수 있음을 의미합니다.

In [57]:
logistics_task = Task(
    description="Coordinate catering and "
                 "equipment for an event "
                 "with {expected_participants} participants "
                 "on {tentative_date}.",
    expected_output="Confirmation of all logistics arrangements "
                    "including catering and equipment setup.",
    human_input=True,
    async_execution=True,
    agent=logistics_manager
)

In [58]:
marketing_task = Task(
    description="Promote the {event_topic} "
                "aiming to engage at least"
                "{expected_participants} potential attendees.",
    expected_output="Report on marketing activities "
                    "and attendee engagement formatted as markdown.",
    async_execution=True,
    output_file="marketing_report.md",  # Outputs the report as a text file
    agent=marketing_communications_agent
)

## 크루 생성하기

**참고**: 물류_작업` 및 `마케팅_작업` 작업에 대해 `async_execution=True`를 설정했으므로 이제 `작업` 목록에서 작업 순서는 중요하지 않습니다.

In [59]:
# Define the crew with agents and tasks
event_management_crew = Crew(
    agents=[venue_coordinator, 
            logistics_manager, 
            marketing_communications_agent],
    
    tasks=[venue_task, 
           logistics_task, 
           marketing_task],
    
    verbose=True
)

2024-05-28 21:51:28,188 - 20496 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


## crew 운영-  실행을 위한 입력을 설정

In [60]:
event_details = {
    'event_topic': "Tech Innovation Conference",
    'event_description': "A gathering of tech innovators "
                         "and industry leaders "
                         "to explore future technologies.",
    'event_city': "San Francisco",
    'tentative_date': "2024-09-15",
    'expected_participants': 500,
    'budget': 20000,
    'venue_type': "Conference Hall"
}

**참고 1**: LLM은 동일한 입력에 대해 서로 다른 출력을 제공할 수 있으므로 동영상에 표시되는 것과 실제 결과가 다를 수 있습니다.

**참고 2**:- 일부 작업에 대해 `human_input=False`를 설정하면 실행이 완료되기 전에 실행이 사용자의 입력을 요청합니다.- 피드백을 요청하면 마우스 포인터를 사용하여 텍스트 상자를 먼저 클릭한 다음 아무 것도 입력하지 마세요.

In [61]:
result = event_management_crew.kickoff(inputs=event_details)

 [DEBUG]: == Working Agent: Venue Coordinator
 [INFO]: == Starting Task: Find a venue in San Francisco that meets criteria for Tech Innovation Conference.


> Entering new CrewAgentExecutor chain...
I need to find a venue in San Francisco that meets the criteria for the Tech Innovation Conference. I should consider the theme, size, and budget constraints when choosing a venue.

Action: Search the internet
Action Input: {"search_query": "tech innovation conference venues in San Francisco"} 


Search results: Title: Top 12 tech conferences in San Francisco for 2024 - TravelPerk
Link: https://www.travelperk.com/blog/top-tech-conferences-in-san-francisco/
Snippet: With over 200 speakers, the San Francisco Tech Summit offers an action-packed agenda for inspiration, insights, innovation, and connections.
---
Title: World Agri-Tech Innovation Summit, March 11-12, 2025
Link: https://worldagritechusa.com/
Snippet: The annual meeting place for the global agtech ecosystem to exchange insights, be

This is the agent final answer: Venue: San Francisco Marriott Marquis
Address: 780 Mission Street, San Francisco, CA 94103
Description: The World Agri-Tech Innovation Summit is hosted at the San Francisco Marriott Marquis, a hotel that is close to shops, restaurants, and entertainment venues. The hotel is easily accessible from local airports.
Nearby Hotels:
1. Hyatt Regency San Francisco Downtown Soma - 4-minute walk
2. Hotel Zelos - 4-minute walk
3. citizenM San Francisco Union Square hotel - 5-minute walk
4. InterContinental San Francisco - 6-minute walk
5. Hotel Zetta - 6-minute walk
6. Hilton Parc 55 - 6-minute walk
7. Hotel Kensington Park - 10-minute walk
8. Inn at Union Square - 10-minute walk
9. Cornell Hotel de France - 12-minute walk
10. San Francisco Proper - 15-minute walk
Parking: The hotel car park has limited availability, with parking spaces allocated on a first-come, first-served basis. The parking fee is $91.2 per day for guests, and valet parking is available.

This

Thought:
I received feedback that the Hilton Hotel looks good as a potential venue for the Tech Innovation Conference. I should gather more information about the Hilton Hotel to confirm if it meets all the criteria for the event.

Action: Read website content
Action Input: {"website_url": "https://www.hilton.com/en/hotels/sfoufhh-hilton-san-francisco-union-square/"} 

Hilton Page Reference Code
Something went wrong
Maybe it’s us, maybe it’s you.(It’s probably us).
Reference No. 18.7dda2317.1716900750.3b523b6
Go Back
or try:
Hilton.com
Customer Service
Global Sites:
العربية
Български
简体中文
繁體中文
Česky
Dansk
Nederlands
Suomi
Deutsch
Français
Italiano
日本語
한국어
Norsk
Polski
Português
România
Pусский
Español
ไทย
Türkçe
Svenska
Hilton
About Hilton Hotels & Resorts
Hilton Affiliates
Site Map
Careers
Development
Media
Shop Our Products at Hilton to Home
RoomKey.com
Web Accessibility
Customer Support
1-800-HILTONS
International (+800 4445 8667)
Hilton Honors Discount Terms & Conditions
Global Priv

Exception in thread Thread-17 (_execute):
Traceback (most recent call last):
  File "E:\miniconda\envs\crewai\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "E:\miniconda\envs\crewai\lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "E:\miniconda\envs\crewai\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "E:\miniconda\envs\crewai\lib\site-packages\crewai\task.py", line 182, in _execute
    result = agent.execute_task(
  File "E:\miniconda\envs\crewai\lib\site-packages\crewai\agent.py", line 221, in execute_task
    result = self.agent_executor.invoke(
  File "E:\miniconda\envs\crewai\lib\site-packages\langchain\chains\base.py", line 163, in invoke
    raise e
  File "E:\miniconda\envs\crewai\lib\site-packages\langchain\chains\base.py", line 153, in invoke
    self._call(inputs, run_manager=run_manager)
  File "E:\miniconda\envs\crewai\lib\site-packages\crewai\agent

Thought: I need to find more information about the Tech Innovation Conference.
Action: Read website content
Action Input: {
  "website_url": "https://www.forrester.com/event/technology-innovation-north-america/"
} 

Technology & Innovation Summit North America - Forrester
 Skip to content 
Overview
 Agenda Speakers Special Programs Awards Sponsors Venue Register 
Toggle Menu
Overview
 Agenda Speakers Special Programs Awards Sponsors Venue Register 
Technology & Innovation SummitNorth America Technology & Data Leaders September 9 – 12, 2024  |  Austin & Digital 
Register 
Unleash The Power Of Tech, Talent, And AI 
We are on the precipice of an exciting AI-led technology revolution. Tech and data leaders must seize the opportunity to play a vital role in delivering business results. Success will require an intentional blend of optimism, skepticism, and pragmatism — especially amid continued economic and geopolitical turmoil and rapidly changing customer and business expectations. Now is 

- 생성된 `venue_details.json` 파일을 표시합니다.

In [63]:
import json
from pprint import pprint

with open('venue_details.json') as f:
    data = json.load(f)

pprint(data)

{'address': "333 O'Farrell St, San Francisco, CA 94102",
 'booking_status': '',
 'capacity': 0,
 'name': 'Hilton San Francisco Union Square'}


- 생성된 `marketing_report.md` 파일을 표시합니다.**참고**: 킥오프` 실행이 성공적으로 실행된 후 `marketing_report.md` 파일이 생성될 때까지 45초 정도 더 기다리세요. 파일이 생성되기 전에 아래 코드를 실행하면 다음과 같은 결과가 출력됩니다:```마케팅_보고서.md```이 출력이 표시되면 조금 더 기다렸다가 다시 시도하세요.

In [64]:
from IPython.display import Markdown
Markdown("marketing_report.md")

The Tech Innovation Conference hosted by Forrester Research is the most suitable event for our company to participate in. The conference is focused on the latest advancements in technology and innovation, which aligns with our company's goals of staying at the forefront of the industry. The conference also provides opportunities for networking and collaboration with other industry leaders, which can be beneficial for our business.

To promote our participation in the conference, we will use a variety of marketing strategies, including:

* Create a dedicated landing page on our website with information about our participation in the conference, including the date, time, and location of the event, as well as a link to register for the event.
* Share information about the conference on our social media channels, using relevant hashtags and keywords to reach our target audience.
* Send out email newsletters to our subscribers with information about the conference and a link to register.
* Place ads on relevant websites and publications to promote our participation in the conference.
* Reach out to industry influencers and bloggers to see if they would be interested in writing about our participation in the conference.

We believe that these marketing strategies will help us to reach our target audience and generate interest in our participation in the Tech Innovation Conference.

# L6: 재무 분석을 위한 멀티 에이전트 협업이 단원에서는 상담원들이 서로 협업할 수 있도록 하는 방법을 배웁니다.

In [65]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- 라이브러리, API 및 LLM 가져오기

In [66]:
from crewai import Agent, Task, Crew

In [67]:
import os
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI

from dotenv import load_dotenv
import os
load_dotenv()

gpt = ChatOpenAI(model= "gpt-3.5-turbo", api_key=os.getenv("OPENAI_API_KEY"))
gemini = ChatGoogleGenerativeAI(model="gemini-pro",google_api_key=os.getenv("GEMINI_API_KEY"))
os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")

## 크루AI 도구

In [68]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## 에이전트 만들기

In [69]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool],
    llm =gpt
)

In [70]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=False,
    tools = [scrape_tool, search_tool],
    llm =gpt
)

In [71]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool],
    llm =gpt
)

In [72]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=False,
    tools = [scrape_tool, search_tool],
    llm =gpt
)

## 작업 만들기

In [95]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [96]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [97]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [98]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## 크루 생성하기
- '프로세스' 클래스는 워크플로를 상담원에게 위임하는 데 도움이 됩니다(직장에서의 매니저처럼).
- 아래 예시에서는 이를 계층적으로 실행합니다.
- 'manager_llm'을 사용하면 사용하려는 "manager" LLM을 선택할 수 있습니다.

In [99]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

2024-05-24 21:29:04,094 - 18520 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


## 크루 운영- 승무원 실행을 위한 입력을 설정합니다.

In [88]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

**참고**: LLM은 동일한 입력에 대해 다른 출력을 제공할 수 있으므로 동영상에 표시되는 것과 실제 결과가 다를 수 있습니다.

In [89]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to delegate the task of continuously monitoring and analyzing market data for AAPL to the Data Analyst to identify trends and predict market movements.

Action: Delegate work to co-worker
Action Input: {"coworker": "Data Analyst", "task": "Continuously monitor and analyze market data for AAPL using statistical modeling and machine learning", "context": "The goal is to identify trends and predict market movements for AAPL. Make sure to provide insights and alerts about significant market opportunities or threats."}


> Entering new CrewAgentExecutor chain...
I need to gather real-time market data and analyze it using statistical modeling and machine learning to provide crucial insights on AAPL's trends and p

KeyboardInterrupt: 

- 최종 결과를 마크다운으로 표시합니다.

In [100]:
from IPython.display import Markdown
Markdown(result)

{
  "name": "Art Deco / Mod / Old Hollywood Inspired Event Venue - Full Venue in the Financial District",
  "address": "Financial District, San Francisco",
  "capacity": 250,
  "booking_status": "$200/hr"
}

# L7: 맞춤형 채용 지원서를 위한 크루 구축이 단원에서는 첫 번째 멀티 에이전트 시스템을 구축합니다.

라이브러리는 이미 교실에 설치되어 있습니다. 개인 컴퓨터에서 이 노트북을 실행하는 경우 다음을 설치할 수 있습니다:```파이썬!.pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29```

In [101]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- 라이브러리, API 및 LLM 가져오기

In [102]:
from crewai import Agent, Task, Crew

**참고**:- 동영상에서는 'gpt-4-turbo'를 사용하지만, 특정 제약 조건으로 인해 이 강좌를 모든 사람에게 무료로 제공하기 위해 여기서 실행할 코드에서는 'gpt-3.5-turbo'를 사용할 것입니다.- 노트북을 _로컬로_ 실행할 때 `gpt-4-turbo`를 사용할 수 있습니다(`gpt-4-turbo`를 사용하면 플랫폼에서 작동하지 않습니다).- 이해해 주셔서 감사합니다!

In [103]:
from dotenv import load_dotenv
import os
from utils import get_openai_api_key, get_serper_api_key
load_dotenv()

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

ImportError: cannot import name 'get_openai_api_key' from 'utils' (E:\miniconda\envs\crewai\lib\site-packages\utils\__init__.py)

## 크루AI 도구

In [104]:
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  SerperDevTool
)

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path='./fake_resume.md')
semantic_search_resume = MDXSearchTool(mdx='./fake_resume.md')

FileNotFoundError: [Errno 2] No such file or directory: './fake_resume.md'

- 노트북에서 `fake_resume.md`를 보려면 아래 셀의 주석을 해제하고 실행하세요.

In [ ]:
# from IPython.display import Markdown, display
# display(Markdown("./fake_resume.md"))

## 에이전트 만들기

In [105]:
# Agent 1: Researcher
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",
    tools = [scrape_tool, search_tool],
    verbose=True,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    )
)

In [106]:
# Agent 2: Profiler
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do increditble research on job applicants "
         "to help them stand out in the job market",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    )
)

NameError: name 'semantic_search_resume' is not defined

In [107]:
# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    )
)

NameError: name 'semantic_search_resume' is not defined

In [ ]:
# Agent 4: Interview Preparer
interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    )
)

## 작업 만들기

In [ ]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=True
)

In [ ]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True
)

- 작업 목록을 작업의 '컨텍스트'로 전달할 수 있습니다.- 그런 다음 작업은 실행 시 해당 작업의 출력을 고려합니다.- 작업은 해당 작업의 출력이 있을 때까지 실행되지 않습니다.

In [ ]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

In [ ]:
# Task for Interview Preparer Agent: Develop Interview Materials
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)


## 크루 생성하기

In [ ]:
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           interview_preparation_task],

    verbose=True
)

## 크루 운영- 승무원 실행을 위한 입력을 설정합니다.

In [ ]:
job_application_inputs = {
    'job_posting_url': 'https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund',
    'github_url': 'https://github.com/joaomdmoura',
    'personal_writeup': """Noah is an accomplished Software
    Engineering Leader with 18 years of experience, specializing in
    managing remote and in-office teams, and expert in multiple
    programming languages and frameworks. He holds an MBA and a strong
    background in AI and data science. Noah has successfully led
    major tech initiatives and startups, proving his ability to drive
    innovation and growth in the tech industry. Ideal for leadership
    roles that require a strategic and innovative approach."""
}

**참고**: LLM은 동일한 입력에 대해 다른 출력을 제공할 수 있으므로 동영상에 표시되는 것과 실제 결과가 다를 수 있습니다.

In [ ]:
### this execution will take a few minutes to run
result = job_application_crew.kickoff(inputs=job_application_inputs)

- 생성된 `tailored_resume.md` 파일을 표시합니다.

In [ ]:
from IPython.display import Markdown, display
display(Markdown("./tailored_resume.md"))

- 생성된 'interview_materials.md' 파일을 표시합니다.

In [ ]:
display(Markdown("./interview_materials.md"))